In [1]:
from datadreamer import DataDreamer
from datadreamer.llms import OpenAI
from datadreamer.steps import DataFromPrompt, ProcessWithPrompt
import json

from src.settings import load_env, DATA_DIR

load_env()

True

In [2]:
with open(DATA_DIR / "sample_legis.txt") as f:
    legis_text = f.read()

legis_text[:100] + " ..."

'Sygnatura akt II AKa 335/15\n\nWYROK\nW IMIENIU RZECZYPOSPOLITEJ POLSKIEJ\nDnia 20 stycznia 2016 r.\n\nSąd ...'

In [33]:
PROMPT = '''\
You are a question-answer generator. Your goal is to generate question-answer pairs given the Context.

Example output: {{"question": "generated question", "answer": "gathered answer"}}

Context: {context}

Step 1: Identify spans that are likely to be answers to questions, identify as many as possible.
Step 2: For each identified span, generate a question.
Step 3: Respond to the question in only a few tokens concisely.
Step 4: Output in JSON format following [...]

Ensure that you distinctly label and delineate Steps 1, 2, 3, and 4.

Let’s think step by step:
'''.format(context=legis_text)

PROMPT

'You are a question-answer generator. Your goal is to generate question-answer pairs given the Context.\n\nExample output: {"question": "generated question", "answer": "gathered answer"}\n\nContext: Sygnatura akt II AKa 335/15\n\nWYROK\nW IMIENIU RZECZYPOSPOLITEJ POLSKIEJ\nDnia 20 stycznia 2016 r.\n\nSąd Apelacyjny we Wrocławiu II Wydział Karny w składzie:\n\nPrzewodniczący: SSA Stanisław Rączkowski (spr.)\n\nSędziowie: SSA Edward Stelmasik\n\nSSO del. do SA Edyta Gajgał\n\nProtokolant: Beata Sienica\n\nprzy udziale prokuratora Prokuratury Apelacyjnej Leszka Pruskiego\n\npo rozpoznaniu w dniu 20 stycznia 2016 r.\n\nsprawy G. K.\n\noskarżonego z art. 280 § 2 kk\n\nna skutek apelacji wniesionej przez oskarżonego\n\nod wyroku Sądu Okręgowego w Jeleniej Górze\n\nz dnia 20 października 2015 r. sygn. akt III K 43/15\n\nI . zmienia zaskarżony wyrok wobec oskarżonego G. K. w ten sposób, że wymierzoną temu oskarżonemu karę obniża do 3 ( trzech) lat pozbawienia wolności,\n\nII. utrzymuje w mocy 

In [34]:
with DataDreamer("./output"):
   gpt_4 = OpenAI(model_name="gpt-4")

   dataset = DataFromPrompt(
      "Legis Q-A Generation",
      args={
         "llm": gpt_4,
         "n": 1,
         "temperature": 1.2,
         "instruction": PROMPT,
      },
      outputs={"generations": "qa_json_list"},
   )
   print(dataset)


[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: ./output
[ 🤖 DataDreamer 💤 ] Step 'Legis Q-A Generation' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'Legis Q-A Generation' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Done. ✨ Results in folder: ./output


DataFromPrompt(
	name='Legis Q-A Generation',
	inputs={},
	args={
		'llm': <OpenAI (gpt-4)>,
		'instruction': 'You are a question-answer generator. Your goal is to generate question-answer pairs given the Context.\n\nExample output: {"question": "generated question", "answer": "gathered answer"}\n\nContext: Sygnatura akt II AKa 335/15\n\nWYROK\nW IMIENIU RZECZYPOSPOLITEJ POLSKIEJ\nDnia 20 stycznia 2016 r.\n\nSąd Apelacyjny we Wrocławiu II Wydział Karny w składzie:\n\nPrzewodniczący: SSA Stanisław Rączkowski (spr.)\n\nSędziowie: SSA Edward Stelmasik\n\nSSO del. do SA Edyta Gajgał\n\nProtokolant: Beata Sienica\n\nprzy udziale prokuratora Prokuratury Apelacyjnej Leszka Pruskiego\n\npo rozpoznaniu w dniu 20 stycznia 2016 r.\n\nsprawy G. K.\n\noskarżonego z art. 280 § 2 kk\n\nna skutek apelacji wniesionej przez oskarżonego\n\nod wyroku Sądu Okręgowego w Jeleniej Górze\n\nz dnia 20 października 2015 r. sygn. akt III K 43/15\n\nI . zmienia zaskarżony wyrok wobec oskarżonego G. K. w ten sposób

In [35]:
print(dataset)

DataFromPrompt(
	name='Legis Q-A Generation',
	inputs={},
	args={
		'llm': <OpenAI (gpt-4)>,
		'instruction': 'You are a question-answer generator. Your goal is to generate question-answer pairs given the Context.\n\nExample output: {"question": "generated question", "answer": "gathered answer"}\n\nContext: Sygnatura akt II AKa 335/15\n\nWYROK\nW IMIENIU RZECZYPOSPOLITEJ POLSKIEJ\nDnia 20 stycznia 2016 r.\n\nSąd Apelacyjny we Wrocławiu II Wydział Karny w składzie:\n\nPrzewodniczący: SSA Stanisław Rączkowski (spr.)\n\nSędziowie: SSA Edward Stelmasik\n\nSSO del. do SA Edyta Gajgał\n\nProtokolant: Beata Sienica\n\nprzy udziale prokuratora Prokuratury Apelacyjnej Leszka Pruskiego\n\npo rozpoznaniu w dniu 20 stycznia 2016 r.\n\nsprawy G. K.\n\noskarżonego z art. 280 § 2 kk\n\nna skutek apelacji wniesionej przez oskarżonego\n\nod wyroku Sądu Okręgowego w Jeleniej Górze\n\nz dnia 20 października 2015 r. sygn. akt III K 43/15\n\nI . zmienia zaskarżony wyrok wobec oskarżonego G. K. w ten sposób

In [36]:
for gen_uttermsg in dataset.output["qa_json_list"]:
    print(gen_uttermsg)
    

Step 1: Identify spans that are likely to be answers to questions.

- "Sąd Apelacyjny we Wrocławiu II Wydział Karny"
- "SA Edyta Gajgał"
- "Leszka Pruskiego"
- "sprawy G. K."
- "art. 280 § 2 kk"
- "20 października 2015 r. sygn. akt III K 43/15"
- "obniża do 3 ( trzech) lat pozbawienia wolności"
- "600 złotych tytułem kosztów nieopłaconej pomocy prawnej"
- "138 zł tytułem zwrotu VAT"

Step 2: For each identified span, generate a question.

- "What was the department involved in judgment?"
- "Who was one of the Judges involved?"
- "Who was the prosecutor involved in the case?"
- "Whose case was this about?"
- "According to which act was the accused blamed for?"
- "When was the first verdict in this case given?"
- "What was the updated sentencing duration on appeal?"
- "What was the fee for the unpaid legal aid provided?"
- "How much VAT was refunded?"

Step 3: Respond to the question in only a few tokens concisely.
  
- "Sąd Apelacyjny we Wrocławiu II Wydział Karny"
- "SA Edyta Gajgał"
-